텐서플로우 2.0 기반 huggingface 라이브러리
KOBERT를 활용한 네이버쇼핑 감성분석

#목차
<b>1) 네이버 쇼핑 데이터 불러오기 및 전처리  
2) BERT 인풋에 맞게 변환  
3) 버트를 활용한 감성분석 모델 생성  
4) 훈련 및 성능 검증  
5) 실제 데이터 적용</b>

# KOBERT 활용한 네이버쇼핑 감성분석 모델

huggingface 패키지를 Colab에 설치

In [1]:
!pip install transformers==2.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.2/660.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=3f43bdc1ff68e5a92db9cc4ae080f370bee5452b6ff043e11684f111d73b0d99
  Stored in directory: /root/.cache/pip/whe

In [2]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.7 MB/s eta 0:00:00


In [3]:
!pip install -q pyyaml h5py

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


텐서플로우 2와 필요한 모듈들을 임포트



In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

구글 드라이브와 Colab을 연동하기

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


사용할 네이버 쇼핑 감상분석 데이터를 다운로드

In [7]:
# 네이버 쇼핑리뷰 감정분석 데이터 다운로드
!git clone https://github.com/bab2min/corpus

Cloning into 'corpus'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 11 (delta 1), pack-reused 0
Receiving objects: 100% (15/15), 11.78 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [8]:
%cd /content/corpus/sentiment
!ls

/content/corpus/sentiment
naver_shopping.txt  README.md  steam.txt


딥러닝 훈련에 사용 할 train 데이터와 test 데이터를 pandas dataframe 형식으로 불러오기

In [62]:
# 파인튜닝용 쇼핑리뷰데이터
df = pd.read_csv("naver_shopping.txt", sep='\t', names=['label', 'document'])
df.shape

(200000, 2)

In [10]:
# 키워드추출용 특정상품리뷰데이터
df_ = pd.read_csv("/content/gdrive/MyDrive/DataScience/data/navershopping_airpod/review_airpod_encoded.csv")
df_.rename(columns = {'star':'label', 'review':'document'}, inplace = True)
# 평점 3점 행 삭제
idx = df_[df_['label']==3].index
df_ = df_.drop(idx)
df_ = df_.drop(['type', 'category'], axis=1)

In [11]:
# 평점을 [1,2] 점은 부정, [4,5] 점은 긍정으로 분류
df['label'] = df['label'].replace(1, 0)
df['label'] = df['label'].replace(2, 0)
df['label'] = df['label'].replace(4, 1)
df['label'] = df['label'].replace(5, 1)

df_['label'] = df_['label'].replace(1, 0)
df_['label'] = df_['label'].replace(2, 0)
df_['label'] = df_['label'].replace(4, 1)
df_['label'] = df_['label'].replace(5, 1)

In [12]:
df

,label,document
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,0,장마라그런가!!! 달지않아요
199996,1,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,1,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,1,넘이쁘고 쎄련되보이네요~


In [13]:
df['label'].value_counts(normalize=True)

0    0.500185
1    0.499815
Name: label, dtype: float64

In [84]:
df_

,grade,review,evaluation
0,5,1인가구입니다 집에 모르는 사람이 방문하는 걸 너무 안 좋아하는 사람이에요매번 생수...,(부정 확률 : 0.96) 부정적인 리뷰입니다.
1,5,SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직접설치 ...,(부정 확률 : 0.86) 부정적인 리뷰입니다.
2,5,☘️ 정수기 상품 도착 ☘️작은 박스에 담겨진 sk 무전원 미니 정수기종이박스 안에...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3,3,┌────── SK매직 미니 Green 41 무전원 정수기 ──────✅ 2022....,(긍정 확률 : 0.98) 긍정적인 리뷰입니다.
4,5,원래 본가는 정수기를 아주 오래전 부터 사용하고 있어서 5년 전 혼자 살기 시작했을...,(부정 확률 : 0.59) 부정적인 리뷰입니다.
5,5,✔ 제품명 : SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직...,(긍정 확률 : 0.85) 긍정적인 리뷰입니다.
6,5,우리집은 이때가지 생수를 배달해서 먹었어요주로 쿠팡에서 배달해서 먹었는데가끔씩 생수...,(부정 확률 : 0.81) 부정적인 리뷰입니다.
7,5,배송이 되었을 때 상자보고 깜짝 놀랐어요상자가 너무 심하게 파손이 되어서요 ㅠㅠ무슨...,(부정 확률 : 0.95) 부정적인 리뷰입니다.
8,5,"집에선 보리차자주 끓여먹고 생수는 정기구독해서 먹었어요.페트병도 많이 나오고, 정...",(부정 확률 : 0.52) 부정적인 리뷰입니다.
9,5,저는 이미 sk매직 정수기를 사용중입니다.그래서 더욱 sk매직 브랜드를 믿고 고르게...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.


In [15]:
# 타겟레이블 분포 확인 타겟레이블 분포 확인
df_['label'].value_counts(normalize=True)
df_['label'].value_counts(normalize=True)

1    0.996672
0    0.003328
Name: label, dtype: float64

In [16]:
# 훈련셋과 검증셋으로 분리
train, test = train_test_split(df,
                               random_state=3,
                               test_size=0.25,
                               stratify=df['label'])

print(f"학습용데이터 : {train.shape}\n테스트용데이터 : {test.shape}")

학습용데이터 : (150000, 2)
테스트용데이터 : (50000, 2)


In [17]:
# 데이터별 타겟 분포 확인
print(train['label'].value_counts(normalize=True))
print(test['label'].value_counts(normalize=True))

0    0.500187
1    0.499813
Name: label, dtype: float64
0    0.50018
1    0.49982
Name: label, dtype: float64


In [18]:
# 훈련셋의 앞부분 출력
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
df_.reset_index(drop=True, inplace=True)
train.head(10)

,label,document
0,0,사용하기불편해요 잘녹지도않고
1,1,사이즈 딱좋아요 휴대성도 좋구요~^^
2,0,오래된 가족 쇼파를 산뜻하게 변화시켜준 것 좋았는데 안고 나면 자꾸 앉은 자리 커버...
3,0,너무약해요 복근은동으로는 좀 ..
4,1,튼튼하고 사이즈도 만족스러워요
5,0,"오프라인보다 저렴한 가격, 빠른 배송은 아주 만족합니다. 하지만 포장은 뜯으니 일부..."
6,0,재구매 첫번째 사진은 비교하기 위해 올려놓았습니다. 두번째사진은 작은 사이즈라 전부...
7,1,깔끔하니 아주이쁘네요~~
8,0,재구매 세번째 구매인데 ㅠㅠ이번엔 상태가 좀 안좋은거 같아요 너무 오래 후숙된 느낌...
9,1,구매가 편해서 좋네요


## 버트 인풋 만들기

한국어 데이터로 훈련되었고, SKT에서 만든 KoBERT를 사용  

https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드 가져옴

In [19]:
pip install sentencepiece

  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [20]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.model_max_length - 2  # take into account special tokens
        self.max_len_sentences_pair = self.model_max_length - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

kobert 토크나이즈를 임포트

In [21]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

loading file tokenizer_78b3253a26.model from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/tokenizer_78b3253a26.model
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/config.json
Model config BertConfig {
  "_name_or_path": "monologg/kobert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Be

tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌  
tokenizer.tokenize => 문장을 토큰화

In [22]:
print(tokenizer.encode("너무 맛있어서 다시 주문했어요"))

[2, 1458, 1967, 7141, 6855, 6553, 1574, 4227, 7874, 3]


In [23]:
print(tokenizer.tokenize("너무 맛있어서 다시 주문했어요"))

['▁너무', '▁맛', '있', '어', '서', '▁다시', '▁주문', '했어요']


네이버 쇼핑 리뷰 문장들을 버트 인풋으로 변환  
문장은 토큰 인풋, 세그먼트 인풋, 마스크 인풋으로 변환  
huggingface에서의 순서 [토큰 인풋, 마스크 인풋, 세그먼트 인풋]

In [24]:
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 64
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

  0%|          | 0/150000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 150000/150000 [01:00<00:00, 2499.79it/s]

In [25]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 50000/50000 [00:11<00:00, 4316.29it/s]


## 버트를 활용한 감성분석 모델 만들기

In [26]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/pytorch_model.bin
PyTorch checkpoint contains 92,186,880 parameters
Loaded 92,186,880 parameters in the TF 2.0 model.
All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [27]:
bert_outputs

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 64, 768) dtype=float32 (created by layer 'tf_bert_model')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [28]:
bert_outputs = bert_outputs[1]

In [29]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [30]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [31]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 64)]                 0         []                            
 )                                                                                                
                                                                                                  
 input_masks (InputLayer)    [(None, 64)]                 0         []                            
                                                                                                  
 input_segment (InputLayer)  [(None, 64)]                 0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   9218688   ['input_word_ids[0][0]',  

## 훈련 및 성능 검증

모델 학습

In [32]:
sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=64, validation_data=(test_x, test_y))

Epoch 1/4
2344/2344 [==============================] - 2253s 947ms/step - loss: 0.2956 - accuracy: 0.8711 - val_loss: 0.1938 - val_accuracy: 0.9317
Epoch 2/4
2344/2344 [==============================] - 2217s 946ms/step - loss: 0.1827 - accuracy: 0.9358 - val_loss: 0.1860 - val_accuracy: 0.9363
Epoch 3/4
2344/2344 [==============================] - 2163s 923ms/step - loss: 0.1487 - accuracy: 0.9496 - val_loss: 0.1819 - val_accuracy: 0.9360
Epoch 4/4
2344/2344 [==============================] - 2216s 945ms/step - loss: 0.1116 - accuracy: 0.9640 - val_loss: 0.2001 - val_accuracy: 0.9334


모델 저장

In [40]:
sentiment_model.save('/content/gdrive/MyDrive/project/project_cs1/model/mymodel')

In [41]:
sentiment_model.save_weights('/content/gdrive/MyDrive/project/project_cs1/model/saveweights')

In [43]:
sentiment_model.load_weights('/content/gdrive/MyDrive/project/project_cs1/model/saveweights')

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


훈련 모델의 예측 성능을 F1 SCORE로 체크하기 위한 작업

In [45]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [46]:
test_set = predict_load_data(test)

  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 50000/50000 [00:12<00:00, 3989.18it/s]


In [47]:
test_set

[array([[   2, 3647, 5377, ...,    1,    1,    1],
        [   2, 3093,  463, ...,    1,    1,    1],
        [   2, 2779, 7836, ...,    1,    1,    1],
        ...,
        [   2, 1185, 7199, ...,    1,    1,    1],
        [   2, 1458,  517, ...,    1,    1,    1],
        [   2, 4379, 7096, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [48]:
preds = sentiment_model.predict(test_set)

1563/1563 [==============================] - 221s 139ms/step


In [49]:
# 부정이면 0, 긍정이면 1 출력
preds

array([[0.00155248],
       [0.00299077],
       [0.99369943],
       ...,
       [0.00154331],
       [0.00205419],
       [0.00155891]], dtype=float32)

우리가 훈련한 모델을 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [50]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     25009
           1       0.93      0.93      0.93     24991

    accuracy                           0.93     50000
   macro avg       0.93      0.93      0.93     50000
weighted avg       0.93      0.93      0.93     50000



In [51]:
import logging
tf.get_logger().setLevel(logging.ERROR)

# 실제 데이터 적용

In [52]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)

    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()

    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value)

In [53]:
review_evaluation_predict("두번 다시 안 살꺼에요")

1/1 [==============================] - 0s 61ms/step
(부정 확률 : 1.00) 부정적인 리뷰입니다.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [54]:
review_evaluation_predict("배송이 좀 느리긴 했어도 품질 자체는 믿을만해요")

1/1 [==============================] - 0s 51ms/step
(긍정 확률 : 0.92) 긍정적인 리뷰입니다.


In [55]:
review_evaluation_predict("디자인이 매우 세련되었으나 쉽게 망가지네요. 내구도가 엉망이에요")

1/1 [==============================] - 0s 51ms/step
(부정 확률 : 1.00) 부정적인 리뷰입니다.


### 에어팟 리뷰데이터 분류

In [56]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)

    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()

    if predict_answer == 0:
      return "(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value)
    elif predict_answer == 1:
      return "(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value

In [85]:
%cd /content/drive/MyDrive/project/project_cs1/coupang_data

/content/drive/MyDrive/project/project_cs1/coupang_data


In [88]:
df_ = pd.read_csv("/content/drive/MyDrive/project/project_cs1/coupang_data/SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직접설치 (1).txt", sep='\t', names=['grade', 'review'])
df_.shape

(50, 2)

In [95]:
df_

,grade,review,evaluation
0,5,1인가구입니다 집에 모르는 사람이 방문하는 걸 너무 안 좋아하는 사람이에요매번 생수...,(부정 확률 : 0.96) 부정적인 리뷰입니다.
1,5,SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직접설치 ...,(부정 확률 : 0.86) 부정적인 리뷰입니다.
2,5,☘️ 정수기 상품 도착 ☘️작은 박스에 담겨진 sk 무전원 미니 정수기종이박스 안에...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3,3,┌────── SK매직 미니 Green 41 무전원 정수기 ──────✅ 2022....,(긍정 확률 : 0.98) 긍정적인 리뷰입니다.
4,5,원래 본가는 정수기를 아주 오래전 부터 사용하고 있어서 5년 전 혼자 살기 시작했을...,(부정 확률 : 0.59) 부정적인 리뷰입니다.
5,5,✔ 제품명 : SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직...,(긍정 확률 : 0.85) 긍정적인 리뷰입니다.
6,5,우리집은 이때가지 생수를 배달해서 먹었어요주로 쿠팡에서 배달해서 먹었는데가끔씩 생수...,(부정 확률 : 0.81) 부정적인 리뷰입니다.
7,5,배송이 되었을 때 상자보고 깜짝 놀랐어요상자가 너무 심하게 파손이 되어서요 ㅠㅠ무슨...,(부정 확률 : 0.95) 부정적인 리뷰입니다.
8,5,"집에선 보리차자주 끓여먹고 생수는 정기구독해서 먹었어요.페트병도 많이 나오고, 정...",(부정 확률 : 0.52) 부정적인 리뷰입니다.
9,5,저는 이미 sk매직 정수기를 사용중입니다.그래서 더욱 sk매직 브랜드를 믿고 고르게...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.


In [89]:
df_['evaluation'] = df_['review'].apply(review_evaluation_predict)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 138ms/step


In [90]:
df_

,grade,review,evaluation
0,5,1인가구입니다 집에 모르는 사람이 방문하는 걸 너무 안 좋아하는 사람이에요매번 생수...,(부정 확률 : 0.96) 부정적인 리뷰입니다.
1,5,SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직접설치 ...,(부정 확률 : 0.86) 부정적인 리뷰입니다.
2,5,☘️ 정수기 상품 도착 ☘️작은 박스에 담겨진 sk 무전원 미니 정수기종이박스 안에...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.
3,3,┌────── SK매직 미니 Green 41 무전원 정수기 ──────✅ 2022....,(긍정 확률 : 0.98) 긍정적인 리뷰입니다.
4,5,원래 본가는 정수기를 아주 오래전 부터 사용하고 있어서 5년 전 혼자 살기 시작했을...,(부정 확률 : 0.59) 부정적인 리뷰입니다.
5,5,✔ 제품명 : SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직...,(긍정 확률 : 0.85) 긍정적인 리뷰입니다.
6,5,우리집은 이때가지 생수를 배달해서 먹었어요주로 쿠팡에서 배달해서 먹었는데가끔씩 생수...,(부정 확률 : 0.81) 부정적인 리뷰입니다.
7,5,배송이 되었을 때 상자보고 깜짝 놀랐어요상자가 너무 심하게 파손이 되어서요 ㅠㅠ무슨...,(부정 확률 : 0.95) 부정적인 리뷰입니다.
8,5,"집에선 보리차자주 끓여먹고 생수는 정기구독해서 먹었어요.페트병도 많이 나오고, 정...",(부정 확률 : 0.52) 부정적인 리뷰입니다.
9,5,저는 이미 sk매직 정수기를 사용중입니다.그래서 더욱 sk매직 브랜드를 믿고 고르게...,(긍정 확률 : 0.99) 긍정적인 리뷰입니다.


In [69]:
df_['review'][0]

'1인가구입니다 집에 모르는 사람이 방문하는 걸 너무 안 좋아하는 사람이에요매번 생수 사먹다가 페트병이 너무 많이 나오고 매번 주문하기 번거로워서 정수기를 많이 찾아보는 중이였어요 그치만 정수기를 설치하려면 기사님께서 방문해야하는데 집에 고양이들도 많이 예민하고모르는 사람이 집에 방문하는걸 극도로 불편해하는 성격이라 늘 차일피일 미뤄왔었습니다 .근데 웬걸 셀프설치 가능한 정수기에다가 수전도 설치 가능한 수전이라 바로 주문했네요 설치는 동영산 보고하는게 제일 좋아요 다만 수전부분에 연결시에 고무패킹 위치가 동영상돠 반대로 되야하는 수전이라 여기서 좀 방황했습니다(물도 역으로 흘러서 싱크대 하단을 다 닦았어요ㅠ고무패킹 이상하다 싶으면 동영상 위치말고 다른쪽에 끼워보세요) 우여곡절 끝에 설치를 끝냈고 단점은 1. 정수기와 연결된 호스가 눈에 보인다는것 (셀프설치에 싱크대에 구멍 안낼려면 어쩔수는 없음)2. 수전을 길게 뽑아서 쓰기가 애매해진다는 점(수전을 길게 빼면 정수기에 연결된 하얀호스도 움직여야하기 때문) 3. 정수기 물을 틀려면 세번의 오픈과정이 필요하다는 점이네요(수전쪽 2개의 열림, 정수기의 열림) 이부분은 사용하다보면 편한쪽으로 적응을 하겠네요 장점은 1.플라스틱이 안나온다! 2.물주문하는 걸 잊고 물이 떨어져 급하게 사러가지 않아도 된다3.요리할때도 맘껏 사용가능하다4.반려동물이 있다면 반려동물도 정수된 물을 맘껏 마실 수 있다5.타인이 방문하지않고 스스로 설치가 가능하다6.정말 저렴한 가격에 정수기를 겟 할수있다1인가구라면 정말 추천합니다 **후기보면서 헷갈렸던 부분처음 배송오면 필터가 장착되어있네요이후의 필터는 교체 시기마다 구매하면 되겠어요(이때부터는 따로 구매)호스가 너무 길어 컷팅이 필요하면 가위말고 칼로 잘라야합니다 그리고 배송박스에 테이핑이 안되어있어서 누구나 바로 오픈이 가능하게 되어있어 오픈된 제품인지 아닌지 알수없어 기분이 좀 묘했네요 이점은 좀 보완했으면 좋겠네--------며칠 사용 후기흠... 집에 공구를 잘 다루시는 분들이 있

In [91]:
# 데이터프레임을 CSV 파일로 저장
df_.to_csv('/content/drive/MyDrive/project/project_cs1/coupang_data/sentimental_output.csv', encoding='utf-8-sig', mode='w')  # index=False를 지정하면 인덱스가 CSV 파일에 저장되지 않습니다.


In [79]:
df_['evaluation'][0]

'(부정 확률 : 0.96) 부정적인 리뷰입니다.'

In [92]:
df_['evaluation'][0].find('부정')

1

In [105]:
import re
index_negative = []
for i in range(50):
    if df_['evaluation'][i] .find('부정') == 1:
        text = df_['evaluation'][i]
        matches = re.findall(r'\d+\.\d+', text)
        if matches:
        # 첫 번째 일치 항목을 부동 소수점(float)으로 변환하여 num 변수에 저장
            num = float(matches[0])
            if num >= 0.80:
                index_negative.append(i)
print(index_negative)

[0, 1, 6, 7, 15, 21, 22, 23, 31, 35, 36, 42]


In [138]:
negative_reviews = []
for i in range(len(index_negative)):
    n = int(index_negative[i])
    negative_reviews.append(df_['review'][n])

In [139]:
import pandas as pd
df = pd.DataFrame(negative_reviews)
print(df)

                                                    0
0   1인가구입니다 집에 모르는 사람이 방문하는 걸 너무 안 좋아하는 사람이에요매번 생수...
1   SK매직 미니 Green41 무전원 정수기 WPU-GBC112 고객직접설치     ...
2   우리집은 이때가지 생수를 배달해서 먹었어요주로 쿠팡에서 배달해서 먹었는데가끔씩 생수...
3   배송이 되었을 때 상자보고 깜짝 놀랐어요상자가 너무 심하게 파손이 되어서요 ㅠㅠ무슨...
4   저희 집은 얼마전까지 6년째 타사의 무전원 직수 정수기를 1년치 필터를  매년 구매...
5   쿠팡의 생수 2번의 연이은 오배송과 상담원의 교환 접수 누락과 회수 처리 누락으로 ...
6   몇 달 전부터 보고 갖고 싶어서 계속 계속 고민만 하다가그냥 큰 맘 먹고 질렀어요....
7   색상은 회백색 이네요 화이트였다면 좀 더 깔끔하고 좋았겠지만 이것도 이것 나름대로 ...
8   ⭐내돈내산⭐️인생 최고의 소비, 별 오천개..! 물고래라 물 진짜 많이먹는데, 돈아...
9   어뎁터 를 줄꺼면 에초에 커플링25미리 어뎁터를 셋트로 주시지  그것만 있으면 어떤...
10  원래 생수 시켜서 먹었는데 아무리 집앞까지 배송해 주시긴 해도 들고 집으로 들어가다...
11  물 사다 먹다보니 비용도 비용이지만 쓰레기가 너무 많이 나와서 항상 불만이었습니다그...


In [140]:
! cd /content/drive/MyDrive/project/project_cs1/coupang_data

In [ ]:
excel_filename = "negative_revews.xlsx"
header_titles = "감성분석_부정리뷰"
df.to_excel("negative_revews.xlsx", index=False, header=)

In [115]:
import re
index_positive = []
for i in range(50):
    if df_['evaluation'][i] .find('긍정') == 1:
        text = df_['evaluation'][i]
        matches = re.findall(r'\d+\.\d+', text)
        if matches:
        # 첫 번째 일치 항목을 부동 소수점(float)으로 변환하여 num 변수에 저장
            num = float(matches[0])
            if num >= 0.98:
                index_positive.append(i)
print(index_positive)

[2, 3, 9, 12, 17, 18, 24, 26, 27, 28, 29, 30, 32, 33, 34, 38, 43, 44, 46, 49]


In [137]:
positive_reviews = []
for i in range(len(index_positive)):
    n = int(index_positive[i])
    positive_reviews.append(df_['review'][n])


In [136]:
import pandas as pd
df = pd.DataFrame(positive_reviews)
print(df)

                                                    0
0   ☘️ 정수기 상품 도착 ☘️작은 박스에 담겨진 sk 무전원 미니 정수기종이박스 안에...
1   ┌────── SK매직 미니 Green 41 무전원 정수기 ──────✅ 2022....
2   저는 이미 sk매직 정수기를 사용중입니다.그래서 더욱 sk매직 브랜드를 믿고 고르게...
3   ✅처음에는 필터 구매하지 않아도 장착되어 같이 배송옵니다!✅냉수, 온수 안나와요! ...
4   배송이 빠르고 안전하게 왔어요. 박스 안에 스티로폼으로 내포장이 한번 더 되어 있어...
5   저희 집은 물은 사먹진 않고 끊여먹고 있는데좋은 기회에 정수기를 사용하게 되어 너무...
6   자가설치로 구매했는데, 싱크대 하단 냉수관에 연결했습니다.설치가 간편하고, 기기 자...
7   이사하고  주방이 넓어져서 정수기 찾다,  발견했어요.첨엔 냉온정수기봤는데,  찬물...
8   다양한 직수 정수기를 알아보다가 가격도 저렴하고 평이 좋아서 주문하게 되었어요이제 ...
9   제품을 최종적으로 설치하고 나니 새삼 편합니다생수를 때마다 안사도 되고 주방에서 내...
10  무타공정수기 꽤 오랜기간동안 찾아보다가 잊어먹고 ㅎㅎㅎ최근에 다시 찾아보기 시작했는...
11  슬림하니 넘 좋습니다.설치는 좀 힘들었지만 하고나니 뿌듯했네요.^^가격대비 고급진 ...
12         혼자 설치하고 확인 결과 만족합니다차후 주번 혼자싱글분께 많이 알려야 겠어요
13  10분 내외로 설치 가능했어요 간단해요. 다만 호스 벨브등 밖으로 좀 지저분하게 노...
14  매번 물을 사 먹는 게 번거로워서 정수기를 구매했습니다. 무전원이라서 전기세도 안 ...
15  수전이 코브라 일체형으로 설치불가라서 주방수전도 입출식으로 사서 남편의 도움으로 바...
16  생수 주문해서 먹다가 sk메직 미니 직접 간단하게 설치해서 잘 먹고 있습니다~픟라스...
17   깔끔한 디자인 맘에 쏙 든답니다.하나더 주